In [1]:
# native packs
import requests
import json
# installed packs
import pandas as pd
from pandas import json_normalize


auth_endpoint = "https://auth.emsicloud.com/connect/token"  # auth endpoint

# replace 'your_client_id' with your client id from your api invite email
client_id = "5f379hywuvh7fvan"
# replace 'your_client_secret' with your client secret from your api invite email
client_secret = "hfCkXQEy"
scope = "emsi_open"  # ok to leave as is, this is the scope we will used

# set credentials and scope
payload = f"client_id={client_id}&client_secret={client_secret}&grant_type=client_credentials&scope={scope}"
# headers for the response
headers = {'content-type': 'application/x-www-form-urlencoded'}
access_token = json.loads((requests.request("POST", auth_endpoint, data=payload, headers=headers)).text)[
    'access_token']  # grabs request's text and loads as JSON, then pulls the access token from that

fields = [
      "id",
      "type",
      "name",
      "isSoftware",
      "infoUrl",
      "tags",
      "isLanguage",
      "description",
      "descriptionSource",
      "category",
      "subcategory"
    ]
fields_str = ','.join(fields)

querystring = { "fields":fields_str}

def fetch_skills_list() -> pd.DataFrame:

    # List of all skills endpoint
    all_skills_endpoint = "https://emsiservices.com/skills/versions/latest/skills"
    # Auth string including access token from above
    auth = f"Authorization: Bearer {access_token}"
    headers = {'authorization': auth}  # headers
    response = requests.request(
        "GET", all_skills_endpoint, headers=headers,params = querystring)  # response
    response = response.json()['data']  # the data

    # all_skills_df = pd.DataFrame(json_normalize(response)); # Where response is a JSON object drilled down to the level of 'data' key
    return response

response = fetch_skills_list()
all_skills_df = pd.DataFrame(json_normalize(response))

In [2]:
import os
os.chdir("..")
os.getcwd()
from py_scripts.db import client

In [11]:
all_skills_df['category.name'].unique()

array(['NULL', 'Information Technology',
       'Maintenance, Repair, and Facility Services',
       'Architecture and Construction', 'Design', 'Engineering',
       'Media and Communications', 'Business', 'Health Care', 'Finance',
       'Human Resources', 'Manufacturing and Production',
       'Marketing and Public Relations', 'Energy and Utilities',
       'Customer and Client Support', 'Education and Training',
       'Public Safety and National Security', 'Science and Research',
       'Analysis', 'Environment',
       'Transportation, Supply Chain, and Logistics',
       'Physical and Inherent Abilities', 'Administration',
       'Law, Regulation, and Compliance', 'Social and Human Services',
       'Sales', 'Performing Arts, Sports, and Recreation',
       'Economics, Policy, and Social Studies',
       'Agriculture, Horticulture, and Landscaping',
       'Personal Care and Services', 'Hospitality and Food Services',
       'Property and Real Estate', nan], dtype=object)

In [13]:
filtred_df = all_skills_df[all_skills_df['category.name'].isin(['Information Technology'])]
len(filtred_df)

5235

In [15]:
import numpy as np 
fields = [
      "id",
      "name",
      "isSoftware",
      "isLanguage",
      "description",
      "category.name",
      "subcategory.name"
    ]
def safe_write(to_write):
    if np.isnan(to_write) :
        return None
    else :
        return to_write

for skill in filtred_df[fields].values : 
    idx,name,isSoftware,isLanguage,description,category,subcategory = skill
    
    meta = {
        'name' : name,
        'category' : category,
        'subCategory' : subcategory,
        'description' : description,
        'isSoftware' : isSoftware , 
        'isLanguage' : isLanguage
    }
    resp = client.create_skill(idx,meta)
    print(resp)
    print('--------')


{'errors': [{'extensions': {'code': 'constraint-violation', 'path': '$.selectionSet.insert_Skill_one.args.object'}, 'message': 'Uniqueness violation. duplicate key value violates unique constraint "Skill_pkey"'}]}
--------
{'errors': [{'extensions': {'code': 'constraint-violation', 'path': '$.selectionSet.insert_Skill_one.args.object'}, 'message': 'Uniqueness violation. duplicate key value violates unique constraint "Skill_pkey"'}]}
--------
{'errors': [{'extensions': {'code': 'constraint-violation', 'path': '$.selectionSet.insert_Skill_one.args.object'}, 'message': 'Uniqueness violation. duplicate key value violates unique constraint "Skill_pkey"'}]}
--------
{'errors': [{'extensions': {'code': 'constraint-violation', 'path': '$.selectionSet.insert_Skill_one.args.object'}, 'message': 'Uniqueness violation. duplicate key value violates unique constraint "Skill_pkey"'}]}
--------
{'errors': [{'extensions': {'code': 'constraint-violation', 'path': '$.selectionSet.insert_Skill_one.args.ob

KeyboardInterrupt: 

In [16]:
5235/60

87.25